In [ ]:
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
import os

In [58]:
def get_subdirectories(folder_path):
    subdirectories = []
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if os.path.isdir(item_path):
            subdirectories.append(item)
    return subdirectories

def get_files_in_subdirectories(folder_path, file_extension):
    files = []
    for root, directories, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith(file_extension):
                files.append(os.path.join(root, filename))
    return files

folder_path = os.getcwd()+'/data/'
print(folder_path)
subdirectories = get_subdirectories(folder_path)
print(subdirectories)

file_extension = ['.csv','.csv.stepMixed']

for i in subdirectories:
    folder_i = folder_path+i
    for j in file_extension:
        files = get_files_in_subdirectories(folder_i,j)
        print(files)




/Users/rodolfocacacho/Documents/Documents/MAI/1.Semester/Artificial Neural Networks and Cognitive Models/Kaggle competition dataset/ANN_Project/data/
['person_8', 'person_7', 'person_12', 'person_5', 'person_2', 'person_3', 'person_4', 'person_11', 'person_10']
['/Users/rodolfocacacho/Documents/Documents/MAI/1.Semester/Artificial Neural Networks and Cognitive Models/Kaggle competition dataset/ANN_Project/data/person_8/ClippedawindaRecording_20230217130617.csv']
['/Users/rodolfocacacho/Documents/Documents/MAI/1.Semester/Artificial Neural Networks and Cognitive Models/Kaggle competition dataset/ANN_Project/data/person_8/awindaRecording_20230217130617.csv.stepMixed']
['/Users/rodolfocacacho/Documents/Documents/MAI/1.Semester/Artificial Neural Networks and Cognitive Models/Kaggle competition dataset/ANN_Project/data/person_7/ClippedawindaRecording_20230206175311.csv', '/Users/rodolfocacacho/Documents/Documents/MAI/1.Semester/Artificial Neural Networks and Cognitive Models/Kaggle competitio